# Tagging paper categories

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

%load_ext autoreload
%autoreload 2

In [4]:
# load environment variables
load_dotenv()

# get API key
api_key = os.getenv("OPENAI_API_KEY")

# init OpenAI client
client = OpenAI(api_key=api_key)


def prompt_gpt(
    model,
    prompt: str,
    temperature: float = 0.0,
    max_tokens: int = 200,
):
    # query ChatGPT
    chat_completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant designed to carefully analyze academic papers based on specific instructions.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return chat_completion.choices[0].message.content

In [6]:
model = "gpt-4o-mini"
prompt = "Say 'this is a test' and nothing else."
response = prompt_gpt(model, prompt)
print(response)

This is a test.


In [23]:
import pandas as pd

data_path = "../data/full-text.csv"
full_df = pd.read_csv(data_path)
# remove .pdf from the title
full_df["title"] = full_df["title"].str.replace(".pdf", "", regex=False)
display(full_df.head())
display(full_df.count())

,title,text,morphology,methodology
0,a comparative study of english to kannada base...,International Journal of Applied Engineering R...,agglutinat,data augmentation
1,"a comparison of transformer, recurrent neural ...",2020 20th International Conference on Advances...,agglutinat,"morphological analyzer, segmentation"
2,a morpheme-based weighting for chinese-mongoli...,"IEICE TRANS. INF. & SYST., VOL.E99–D, NO.11 NO...",NaN,segmentation
3,a morphological analyser for maltese,ScienceDirect\nAvailable online at www.science...,fusional,morphological analyzer
4,a multi-agent solution for managing complexity...,"B. Hettige et al., Int. J. of Design & Nature ...",NaN,morphological analyzer


title          177
text           177
morphology     125
methodology    150
dtype: int64

In [24]:
murilo_df = pd.read_csv("../data/murilo_titles.csv")
display(murilo_df.head())
display(murilo_df.count())

,title
0,a morphological analyser for maltese
1,a novel morphological analysis based approach ...
2,a systematic analysis of subwords and cross-li...
3,an evaluation of subword segmentation strategi...
4,analysis of subword tokenization for transform...


title    47
dtype: int64

In [25]:
# filter the dataframe to only include the titles in murilo_titles
filtered_df = full_df[full_df["title"].isin(murilo_df["title"])]
display(filtered_df.head())
display(filtered_df.count())

,title,text,morphology,methodology
3,a morphological analyser for maltese,ScienceDirect\nAvailable online at www.science...,fusional,morphological analyzer
7,a novel morphological analysis based approach ...,(IJACSA) International Journal of Advanced Com...,NaN,NaN
10,a systematic analysis of subwords and cross-li...,Findings of the Association for Computational ...,"analytic, agglutinat","subword modeling, segmentation"
17,analysis of subword tokenization for transform...,\n979-8-3503-5362-4/24/$31.00 ©2024 IEEE Ana...,agglutinat,"subword modeling, segmentation"
27,chinese-uyghur bilingual lexicon induction bas...,Chinese-Uyghur Bilingual Lexicon Induction Ba...,NaN,"morphological analyzer, segmentation, data aug..."


title          40
text           40
morphology     32
methodology    35
dtype: int64

In [ ]:
# RQs
research_questions = """
    1. What challenges do varying degrees of morphological complexity (e.g., isolating, fusional, agglutinative, and polysynthetic languages) pose to machine translation systems in a low-resource language context?
    2. What techniques (e.g., rule-based methods, statistical models, or neural architectures) have been proposed to address these challenges?
    3. How do morphology-aware techniques (e.g. subword modeling, morphological analyzers) compare in effectiveness for low-resource machine translation?
    4. What are the specific findings, challenges, and proposed solutions and results for machine translation of languages in each different morphological typology (polysynthetic, agglutinative, fusional)?
"""

In [ ]:
# get the first paper
text = filtered_df["text"].iloc[0]

prompt = f"""
Based on the following paper, answer the following research questions with as much detail as possible and using highly technical language.

RESEARCH QUESTIONS:
{research_questions}

PAPER:
{text}

Only return the answers to each research question in the following format:
RESEARCH QUESTIONS ANSWERS:
1. Research Question 1 Answer.
2. Research Question 2 Answer.
3. Research Question 3 Answer.
4. Research Question 4 Answer.
"""

In [62]:
model = "gpt-4o-mini"
response = prompt_gpt(model, prompt, max_tokens=2000)
response

"RESEARCH QUESTIONS ANSWERS:\n1. The varying degrees of morphological complexity in languages, such as isolating, fusional, agglutinative, and polysynthetic typologies, present significant challenges to machine translation systems, particularly in low-resource language contexts. Isolating languages, characterized by minimal inflection, may lead to a lack of morphological information, complicating the translation process. In contrast, fusional languages, which combine multiple grammatical categories into single morphemes, can result in ambiguity and difficulty in disambiguating forms. Agglutinative languages, with their extensive use of affixes, may overwhelm translation systems with numerous morphological variations, while polysynthetic languages, which can express complex ideas in single words through extensive affixation, pose challenges in capturing the full semantic and syntactic nuances. The limited availability of training data exacerbates these issues, as machine translation sys

In [63]:
# extract the answers from the response
response_answers = response.split("\n")
response_answers = [
    ans
    for ans in response_answers
    if ans.strip() and not ans.startswith("RESEARCH QUESTIONS ANSWERS:")
]
response_answers

['1. The varying degrees of morphological complexity in languages, such as isolating, fusional, agglutinative, and polysynthetic typologies, present significant challenges to machine translation systems, particularly in low-resource language contexts. Isolating languages, characterized by minimal inflection, may lead to a lack of morphological information, complicating the translation process. In contrast, fusional languages, which combine multiple grammatical categories into single morphemes, can result in ambiguity and difficulty in disambiguating forms. Agglutinative languages, with their extensive use of affixes, may overwhelm translation systems with numerous morphological variations, while polysynthetic languages, which can express complex ideas in single words through extensive affixation, pose challenges in capturing the full semantic and syntactic nuances. The limited availability of training data exacerbates these issues, as machine translation systems rely heavily on large c